In [17]:
import json
from collections import Counter
from keras.models import Model
from keras.layers import Embedding, Input, Reshape
from keras.layers.merge import Dot
from sklearn.linear_model import LinearRegression
import numpy as np
import random
import os
from sklearn import svm
from keras.utils.vis_utils import plot_model

In [18]:
my_path=os.getcwd()+"/"
with open(my_path+'data/wp_movies_10k.ndjson') as fin:
    movies = [json.loads(l) for l in fin]

In [19]:

"""
movies是一个全量数据的数组
其中的每个元素movie，形如下面。其中
第1个元素为电影名;
第2个元素是字典;
第3个元素是数组，包括很多实体名；
第4个元素一个百分率；
第5个元素是日期。
['Deadpool (film)', 
 {'gross': '$783.1 million', 'country': 'United States', 'runtime': '108 minutes', 'caption': 'Theatrical release poster', 'budget': '$58 million', 'Software Used': 'Adobe Premier Pro', 'cinematography': 'Ken Seng', 'director': 'Tim Miller', 'language': 'English', 'editing': 'Julian Clarke', 'alt': "Official poster shows the titular hero Deadpool standing in front of the viewers, with hugging his hands, and donning his traditional black and red suit and mask, and the film's name, credits and billing below him.", 'music': 'Tom Holkenborg', 'name': 'Deadpool', 'image': 'Deadpool poster.jpg', 'distributor': '20th Century Fox'}, 
 ['Tim Miller (director)', 'Simon Kinberg', 'Ryan Reynolds', 'Lauren Shuler Donner', 'Rhett Reese', 'Paul Wernick', 'Deadpool', 'Fabian Nicieza', 'Rob Liefeld', 'Morena Baccarin', 'Ed Skrein', 'T.J. Miller', 'Gina Carano', 'Leslie Uggams', 'Brianna Hildebrand', 'Stefan Kapičić', 'Junkie XL', 'Julian Clarke', 'Marvel Entertainment', 'Kinberg Genre', 'Lauren Shuler Donner', 'TSG Entertainment', '20th Century Fox', 'Le Grand Rex', 'Variety (magazine)', 'Box Office Mojo', 'superhero film', 'Tim Miller (director)', 'Rhett Reese', 'Paul Wernick', 'Marvel Comics', 'Deadpool', 'X-Men (film series)', 'Ryan Reynolds', 'Morena Baccarin', 'Ed Skrein', 'T.J. Miller', 'Gina Carano', 'Leslie Uggams', 'Brianna Hildebrand', 'Stefan Kapičić', 'antihero', 'New Line Cinema', '20th Century Fox', 'X-Men Origins: Wolverine', 'principal photography', 'Vancouver', 'IMAX', 'Digital Light Processing', 'D-Box Technologies', 'List of accolades received by Deadpool (film)', 'Golden Globe Award', 'Golden Globe Award for Best Motion Picture – Musical or Comedy', 'Golden Globe Award for Best Actor – Motion Picture Musical or Comedy', 'Producers Guild of America Award', "Critics' Choice Movie Awards", "Critics' Choice Movie Award for Best Comedy", "Critics' Choice Movie Award for Best Actor in a Comedy", '2016 in film', '#Sequels', 'nonlinear narrative', 'Deadpool', 'special forces', 'Copycat (Marvel Comics)', 'Liver cancer', 'Lung cancer', 'Prostate cancer', 'Brain tumor', 'Ajax (comics)#Deadpool character', 'Angel Dust (comics)', 'healing factor', 'rebar', 'Weasel (Marvel Comics)', 'Blind Al', 'Colossus (comics)', 'Negasonic Teenage Warhead', 'X-Men', 'helicarrier', 'post-credits scene', 'Cable (comics)', 'File:Cast of Deadpool.jpg', 'San Diego Comic-Con', 'Ryan Reynolds', 'Deadpool', 'cancer', 'X-Men Origins: Wolverine', 'fourth wall', 'Morena Baccarin', 'Copycat (Marvel Comics)', 'damsel in distress', 'Ed Skrein', 'Ajax (comics)#Deadpool character', 'Weapon X', 'YouTube', 'Tim Miller (director)', 'T.J. Miller', 'Weasel (Marvel Comics)', 'Simon Kinberg', 'USA Today', 'Gina Carano', 'Angel Dust (comics)', 'Leslie Uggams', 'Blind Al', 'Indiewire', 'Brianna Hildebrand', 'Negasonic Teenage Warhead', 'X-Men', 'Marvel Studios', 'Kevin Feige', 'Ego the Living Planet', 'Guardians of the Galaxy Vol. 2', 'Stefan Kapičić', 'Colossus (comics)', 'Daniel Cudmore', 'X2 (film)', 'X-Men: The Last Stand', 'X-Men: Days of Future Past', 'Andre Tricoteux', 'Karan Soni', 'Jed Rees', 'Agent Smith', 'Stan Lee', 'Rob Liefeld', 'Isaac C. Singleton Jr.', 'Bob, Agent of Hydra', 'Hydra (comics)', 'Nathan Fillion', 'Twitter', 'Simon Kinberg', 'Artisan Entertainment', 'Marvel Entertainment', 'Deadpool', 'New Line Cinema', 'David S. Goyer', 'Ryan Reynolds', 'Shar Pei', 'Cable & Deadpool', 'Turnaround (filmmaking)', 'X-Men Origins: Wolverine', 'Lauren Shuler Donner', 'Reboot (fiction)', 'fourth wall', 'Rhett Reese', 'Paul Wernick', 'Robert Rodriguez', 'Tim Miller (director)', 'Adam Berg (director)', 'Blur Studio', 'James Cameron', 'David Fincher', 'development hell', 'Garrison Kane', 'Cannonball (comics)', 'Computer-generated imagery', 'Hillbilly', 'Wyre (comics)', 'Angel Dust (comics)', 'Cable (comics)', 'List of directorial debuts', 'Colossus (comics)', 'Simon Kinberg', 'The Hollywood Reporter', 'T. J. Miller', 'Ed Skrein', 'Gina Carano', 'Angel Dust (comics)', 'Morena Baccarin', 'Taylor Schilling', 'Crystal Reed', 'Rebecca Rittenhouse', 'Sarah Greene (actress)', 'Jessica De Gouw', 'Weasel (Marvel Comics)', 'Copycat (Marvel Comics)', 'Brianna Hildebrand', 'Negasonic Teenage Warhead', 'Ajax (comics)#Deadpool character', 'Leslie Uggams', 'Blind Al', 'Jed Rees', 'Stefan Kapičić', 'Colossus (comics)', 'Daniel Cudmore', 'Twitter', 'File:Deadpool, Georgia Viaduct, Vancouver, April 6 2015 - 3.jpg', 'Rolling Stone', 'Principal photography', 'Vancouver', 'stunt coordinator', 'CBC News', 'David Cronenberg', 'Eastern Promises', 'Yahoo! Movies', 'Digital Domain', 'Weta Digital', 'Rodeo FX', 'Luma Pictures', 'Image Engine', 'Adobe Systems', 'matte paintings', 'helicarrier', 'Detroit', 'Chicago', 'File:Colossus - mocap.jpg', 'Colossus (comics)', 'Computer-generated imagery', 'Digital Domain', 'Cold-formed steel', 'Hot working', 'Taxicab', 'Junkie XL', 'ARP 2600', 'Synclavier', 'Oberheim Electronics#Oberheim polyphonic synthesizers', 'io9', 'YouTube personality', 'Deadpool (video game)', 'Milan Records', 'Grand Rex', 'IMAX', 'Digital Light Processing', 'D-Box Technologies', 'The Hollywood Reporter', 'Uzbekistan', 'Central Board of Film Certification', 'The Hollywood Reporter', 'Hong Kong', 'Singapore', 'standing ovation', 'The Hollywood Reporter', 'Meta-joke', 'Business Insider', 'viral marketing', 'Christmas', "Valentine's Day", 'io9', 'emoji', 'YouTube', 'Screen Junkies', 'The Guardian', 'Blu-ray', '2016 in film', 'The Matrix Reloaded', 'Forbes', 'Variety (magazine)', 'X-Men: Days of Future Past', 'Deadline.com', 'James Cameron', 'George Lucas', 'Star Wars: Episode III – Revenge of the Sith', 'The Dark Knight Rises', 'The Hollywood Reporter', 'IMAX', '3D film', 'Los Angeles Times', 'Yahoo!', 'Variety (magazine)', 'Forbes (magazine)', "Presidents' Day (United States)", 'Zoolander 2', 'How to Be Single', 'The Hollywood Reporter', 'Deadline.com', 'Variety (magazine)', 'Guardians of the Galaxy (film)', 'Captain America: The Winter Soldier', 'The Hollywood Reporter', 'Deadline.com', 'R-rated', 'The Hangover Part II', 'Fifty Shades of Grey (film)', 'The Hollywood Reporter', 'Deadline.com', 'word of mouth', 'Star Wars: Episode III – Revenge of the Sith', 'Forbes', 'Century Theatres', 'Deadline.com', 'The Hollywood Reporter', 'The Avengers (2012 film)', 'The Hunger Games (film)', 'Forbes (magazine)', 'Furious 7', 'Alice in Wonderland (2010 film)', 'Wanted (2008 film)', 'Watchmen (film)', 'The Vow (2012 film)', 'X-Men (film)', 'X-Men: First Class', 'The Wolverine (film)', 'X-Men Origins: Wolverine', 'Deadline.com', 'Second weekend in box office performance', 'Avengers: Age of Ultron', 'X-Men: The Last Stand', '300 (film)', 'Forbes', 'The Passion of the Christ', 'Forbes', 'Zootopia', 'London Has Fallen', 'Non-Hispanic whites', 'Hispanic', 'African-American', 'Asian Americans', 'Guardians of the Galaxy (film)', 'Captain America: Civil War', 'Avengers: Age of Ultron', 'Batman v Superman: Dawn of Justice', 'Suicide Squad (film)', 'The Hollywood Reporter', 'Spectre (2015)', 'United Kingdom and Ireland', 'Chinese New Year', 'Hong Kong', 'Singapore', 'The Mermaid (2016 film)', 'Deadline.com', 'Ip Man 3', 'Zootopia', 'Iron Man 3', 'Star Wars: The Force Awakens', 'Rotten Tomatoes', 'Rotten Tomatoes', 'Metacritic', 'CinemaScore', 'Peter Travers', 'Rolling Stone', 'Rolling Stone', 'TheWrap', 'Alonso Duralde', 'Guardians of the Galaxy (film)', 'Christy Lemire', 'Richard Roeper', 'Chicago Sun-Times', 'Kenneth Turan', 'Los Angeles Times', 'Los Angeles Times', 'The Hollywood Reporter', 'Deadline.com', 'Spider-Man', 'Superhero Hype', '/Film', 'Domino (comics)', 'TheWrap', 'Mashable', 'David Leitch (filmmaker)', 'Rupert Sanders', 'Drew Goddard', 'Mashable', 'X-Force', 'Hugh Jackman', 'Twitter', 'Variety (magazine)', 'The Hollywood Reporter', 'British Board of Film Classification', 'Los Angeles Times', 'Variety (magazine)', 'MTV News', 'Empire (film magazine)', 'The Hollywood Reporter', 'Variety (magazine)', 'The Daily Dot', 'Bleeding Cool', 'The Hollywood Reporter', 'TheWrap', 'The Hollywood Reporter', 'Variety (magazine)', 'Entertainment Weekly', 'Deadline.com', 'MTV News', 'Deadline.com', 'Business Insider', 'The Hollywood Reporter', 'Twitter', 'Empire (magazine)', 'fxguide', 'Category:20th Century Fox films', 'Category:2010s action films', 'Category:2010s comedy films', 'Category:2010s superhero films', 'Category:2016 films', 'Category:American action comedy films', 'Category:American black comedy films', 'Category:American films', 'Category:Deadpool', 'Category:Directorial debut films', 'Category:English-language films', 'Category:Film scores by Junkie XL', 'Category:Film spin-offs', 'Category:Films about cancer', 'Category:Films about revenge', 'Category:Films directed by Tim Miller', 'Category:Films set in New York', 'Category:Films shot in Vancouver', 'Category:Films with live action and animation', 'Category:Human experimentation in fiction', 'Category:IMAX films', 'Category:Metafictional works', 'Category:Nonlinear narrative films', 'Category:Performance capture in film', 'Category:Self-reflexive films', 'Category:Superhero comedy films', 'Category:Vigilante films', 'Category:X-Men films'], 
 '84%', 
 '6.9/10']
"""
link_counts = Counter()
for movie in movies[:20]:
    # print(movie)
    # print("#"*20)
    # link_counts是一个计数器对象，将统计所有实体名的出现次数。
    link_counts.update(movie[2])
# 显示出现频率最高的10个实体名。
link_counts.most_common(10)

[('Deadline.com', 167),
 ('The Hollywood Reporter', 167),
 ('Variety (magazine)', 139),
 ('Forbes', 91),
 ('Marvel Comics', 54),
 ('Los Angeles Times', 46),
 ('Rotten Tomatoes', 44),
 ('Metacritic', 44),
 ('Entertainment Weekly', 43),
 ('The Guardian', 38)]

In [20]:
# top_links数组，元素为所有出现三次以上的链接名
top_links = [link for link, c in link_counts.items() if c >= 3]
# link_to_idx是一个字典，用于存下所有连接名对应的top_links索引
# 即top_links[link_to_idx[链接名]]即可拿到movie_to_idx[电影名]对应的movie元素
link_to_idx = {link: idx for idx, link in enumerate(top_links)}
# movie_to_idx是一个字典，用于存下所有电影名对应的movies索引
# 即movies[movie_to_idx[电影名]]即可拿到movie_to_idx[电影名]对应的movie元素
movie_to_idx = {movie[0]: idx for idx, movie in enumerate(movies)}
pairs = []
for movie in movies:
    # 对于每一个movie，遍历其链接名
    # 如果该链接名存在于link_to_idx中，就把该链接名的top_links索引，该电影名在movies中的索引，拼成元组，存下来。
    pairs.extend((link_to_idx[link], movie_to_idx[movie[0]]) for link in movie[2] if link in link_to_idx)
print(pairs[:10])
pairs_set = set(pairs)
# len(pairs)  总的索引对名，140940个
# len(top_links),总的连接名 683个（指的是大于3次的高频链接）
# len(movie_to_idx) 总共的电影名 10000个
len(pairs), len(top_links), len(movie_to_idx)

[(279, 0), (612, 0), (358, 0), (322, 0), (461, 0), (258, 0), (504, 0), (401, 0), (288, 0), (1, 0)]


(140940, 683, 10000)

In [21]:
def movie_embedding_model(embedding_size=50):
    link = Input(name='link', shape=(1,))
    movie = Input(name='movie', shape=(1,))
    link_embedding = Embedding(name='link_embedding', 
                               input_dim=len(top_links), 
                               output_dim=embedding_size)(link)
    movie_embedding = Embedding(name='movie_embedding', 
                                input_dim=len(movie_to_idx), 
                                output_dim=embedding_size)(movie)
    dot = Dot(name='dot_product', normalize=True, axes=2)([link_embedding, movie_embedding])
    merged = Reshape((1,))(dot)
    model = Model(inputs=[link, movie], outputs=[merged])
    model.compile(optimizer='nadam', loss='mse')
    return model

model = movie_embedding_model()
plot_model(model, to_file='movie_embedding_model.png',show_shapes=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
link_embedding (Embedding)      (None, 1, 50)        34150       link[0][0]                       
__________________________________________________________________________________________________
movie_embedding (Embedding)     (None, 1, 50)        500000      movie[0][0]                      
__________________________________________________________________________________________________
dot_produc

In [22]:
random.seed(5)

# 可参考博客 https://blog.csdn.net/ChenVast/article/details/83023326 的解释
def batchifier(pairs, positive_samples=50, negative_ratio=10):
    # 50个正样本，550个总样本（batch_size），推出500个负样本。
    batch_size = positive_samples * (1 + negative_ratio)
    print('batch_size=',batch_size)
    # batch_size行，3列。
    batch = np.zeros((batch_size, 3))
    while True:
        # 在pairs数组中，随机采样50个样本作为正例，并把二元组扩充成三元组，第三元为label，标为1
        for idx, (link_id, movie_id) in enumerate(random.sample(pairs, positive_samples)):
            batch[idx, :] = (link_id, movie_id, 1)
        # 原来是idx = positive_samples，改成下式更好理解。
        idx = idx+1
        
        # 找出batch_size-positive_samples=500个负样本，
        while idx < batch_size:
            movie_id = random.randrange(len(movie_to_idx))
            link_id = random.randrange(len(top_links))
            if not (link_id, movie_id) in pairs_set:
                batch[idx, :] = (link_id, movie_id, -1)
                # 只有(link_id, movie_id)不在pairs_set中，才会idx才会累加。因此一定能找够500个负样本。
                idx += 1
        np.random.shuffle(batch)
        # 第一个元素为feature字典，第二个元素为label。
        # batch[:, i]是对第i列的读取。而之前写入batch[idx, :] 是对第idx行的写入。
        yield {'link': batch[:, 0], 'movie': batch[:, 1]}, batch[:, 2]

next(batchifier(pairs, positive_samples=3, negative_ratio=2))

batch_size= 9


({'link': array([ 646.,  664.,  380.,  476.,  160.,  286.,  556.,  587.,  651.]),
  'movie': array([ 3940.,  4080.,  1854.,   475.,   849.,  2040.,  6238.,  1670.,
          9613.])},
 array([ 1., -1., -1., -1., -1.,  1., -1., -1.,  1.]))

In [23]:
positive_samples_per_batch = 512

model.fit_generator(
    batchifier(pairs, positive_samples=positive_samples_per_batch, negative_ratio=10),
    epochs=15,
    steps_per_epoch=len(pairs) // positive_samples_per_batch,
    verbose=2
)

batch_size= 5632
Epoch 1/15
 - 9s - loss: 0.4160
Epoch 2/15
 - 8s - loss: 0.1992
Epoch 3/15
 - 8s - loss: 0.1940
Epoch 4/15
 - 8s - loss: 0.1957
Epoch 5/15
 - 8s - loss: 0.1942
Epoch 6/15
 - 8s - loss: 0.1852
Epoch 7/15
 - 8s - loss: 0.1858
Epoch 8/15
 - 8s - loss: 0.1840
Epoch 9/15
 - 8s - loss: 0.1854
Epoch 10/15
 - 8s - loss: 0.1845
Epoch 11/15
 - 8s - loss: 0.1911
Epoch 12/15
 - 8s - loss: 0.1915
Epoch 13/15
 - 8s - loss: 0.1937
Epoch 14/15
 - 8s - loss: 0.1928
Epoch 15/15
 - 8s - loss: 0.1918


In [24]:
movie = model.get_layer('movie_embedding')
movie_weights = movie.get_weights()[0]
movie_lengths = np.linalg.norm(movie_weights, axis=1)
normalized_movies = (movie_weights.T / movie_lengths).T

def similar_movies(movie):
    dists = np.dot(normalized_movies, normalized_movies[movie_to_idx[movie]])
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c, movies[c][0], dists[c])

similar_movies('Rogue One')

29 Rogue One 1.0
143 Iron Man 3 0.899117
594 Dredd 0.886196
3349 Star Wars: The Force Awakens 0.886008
154 Star Trek (film) 0.882786
86 Tomorrowland (film) 0.881801
22 Jurassic World 0.875209
34 Doctor Strange (film) 0.874467
398 Ben-Hur (2016 film) 0.871439
116 Guardians of the Galaxy Vol. 2 0.869713


In [25]:
link = model.get_layer('link_embedding')
link_weights = link.get_weights()[0]
link_lengths = np.linalg.norm(link_weights, axis=1)
normalized_links = (link_weights.T / link_lengths).T

def similar_links(link):
    dists = np.dot(normalized_links, normalized_links[link_to_idx[link]])
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c, top_links[c], dists[c])

similar_links('George Lucas')

261 George Lucas 1.0
335 Jaws (film) 0.990468
63 Star Wars 0.988193
101 Mark Hamill 0.97913
257 Princess Leia 0.977938
323 American Society of Cinematographers 0.977316
263 American Cinematographer 0.976752
391 James Cameron 0.976021
88 List of highest-grossing films 0.974867
465 Category:Space adventure films 0.974092


In [26]:
best = ['Star Wars: The Force Awakens', 'The Martian (film)', 'Tangerine (film)', 'Straight Outta Compton (film)',
        'Brooklyn (film)', 'Carol (film)', 'Spotlight (film)']
worst = ['American Ultra', 'The Cobbler (2014 film)', 'Entourage (film)', 'Fantastic Four (2015 film)',
         'Get Hard', 'Hot Pursuit (2015 film)', 'Mortdecai (film)', 'Serena (2014 film)', 'Vacation (2015 film)']
y = np.asarray([1 for _ in best] + [0 for _ in worst])
X = np.asarray([normalized_movies[movie_to_idx[movie]] for movie in best + worst])
X.shape

(16, 50)

In [27]:
clf = svm.SVC(kernel='linear')
clf.fit(X, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [28]:
estimated_movie_ratings = clf.decision_function(normalized_movies)
best = np.argsort(estimated_movie_ratings)
print('best:')
for c in reversed(best[-5:]):
    print(c, movies[c][0], estimated_movie_ratings[c])

print('worst:')
for c in best[:5]:
    print(c, movies[c][0], estimated_movie_ratings[c])


best:
6 The Martian (film) 1.48614916127
18 Star Wars (film) 1.41036292161
1 The Revenant (2015 film) 1.34714493871
149 12 Years a Slave (film) 1.34635203097
249 Zero Dark Thirty 1.30857691226
worst:
1594 Marvel Animated Features -2.00419661771
5248 The Death and Return of Superman (film) -1.99803400329
2583 Wonder Woman (2009 film) -1.90871988026
136 Justice League vs. Teen Titans -1.90354707261
590 Batman vs. Robin -1.8804592811


In [29]:
rotten_y = np.asarray([float(movie[-2][:-1]) / 100 for movie in movies if movie[-2]])
rotten_X = np.asarray([normalized_movies[movie_to_idx[movie[0]]] for movie in movies if movie[-2]])

In [30]:
TRAINING_CUT_OFF = int(len(rotten_X) * 0.8)
regr = LinearRegression()
regr.fit(rotten_X[:TRAINING_CUT_OFF], rotten_y[:TRAINING_CUT_OFF])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [31]:
error = (regr.predict(rotten_X[TRAINING_CUT_OFF:]) - rotten_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 0.08'

In [32]:
error = (np.mean(rotten_y[:TRAINING_CUT_OFF]) - rotten_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 0.09'

In [33]:
def gross(movie):
    v = movie[1].get('gross')
    if not v or not ' ' in v:
        return None
    v, unit = v.split(' ', 1)
    unit = unit.lower()
    if not unit in ('million', 'billion'):
        return None
    if not v.startswith('$'):
        return None
    try:
        v = float(v[1:])
    except ValueError:
        return None
    if unit == 'billion':
        v *= 1000
    return v

movie_gross = [gross(m) for m in movies]
movie_gross = np.asarray([gr for gr in movie_gross if gr is not None])
highest = np.argsort(movie_gross)[-10:]
for c in reversed(highest):
    print(c, movies[c][0], movie_gross[c])

6 The Martian (film) 10900.0
7 List of Marvel Cinematic Universe films 4300.0
49 Back to the Future 3900.0
71 The Conjuring 2932.0
162 Thor (film) 2464.0
36 Furious 7 2340.0
30 Finding Dory 2187.0
1906 Jane Eyre (2011 film) 2068.0
19 Interstellar (film) 1670.0
2251 An American Werewolf in London 1655.0


In [34]:
gross_y = np.asarray([gr for gr in movie_gross if gr])
gross_X = np.asarray([normalized_movies[movie_to_idx[movie[0]]] for movie, gr in zip(movies, movie_gross) if gr])

In [35]:
TRAINING_CUT_OFF = int(len(gross_X) * 0.8)
regr = LinearRegression()
regr.fit(gross_X[:TRAINING_CUT_OFF], gross_y[:TRAINING_CUT_OFF])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [36]:
error = (regr.predict(gross_X[TRAINING_CUT_OFF:]) - gross_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 11453.83'

In [37]:
error = (np.mean(gross_y[:TRAINING_CUT_OFF]) - gross_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 14115.59'